In [1]:
#Importing libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
import apimoex

from data_loader import index_history
from data_loader import ticker_prices, get_ticker_history
from pypfopt import expected_returns, risk_models, EfficientFrontier, objective_functions
from optimizer import optimizer_for_tickers

import warnings
warnings.filterwarnings('ignore')

In [56]:
import os
import requests
import pandas as pd
import sqlite3
from time import sleep


def fetch_moex_history(ticker, start=0):
    """
    Получает данные истории торгов с MOEX для заданного тикера.
    Пагинация через параметр `start`
    """
    url = f"https://iss.moex.com/iss/history/engines/stock/markets/shares/securities/{ticker}.json"
    params = {
        "start": start
    }
    response = requests.get(url, params=params, timeout=10)
    response.raise_for_status()
    data = response.json()

    columns = data['history']['columns']
    rows = data['history']['data']
    df = pd.DataFrame(rows, columns=columns)
    return df


def get_full_history(ticker):
    """
    Загружает всю доступную историю для тикера с пагинацией.
    """
    all_data = []
    start = 0

    while True:
        df = fetch_moex_history(ticker, start)
        if df.empty:
            break
        all_data.append(df)
        start += 100
        sleep(0.5)  # чтобы не заддосить MOEX

    full_df = pd.concat(all_data, ignore_index=True)
    return full_df


def save_to_csv(df, ticker):
    os.makedirs("data", exist_ok=True)
    filepath = f"data/{ticker}.csv"
    df.to_csv(filepath, index=False)
    print(f"Данные сохранены в {filepath}")


def save_to_sqlite(df, db_path="moex_data.db", table_name=None):
    if table_name is None:
        table_name = "history"
    conn = sqlite3.connect(db_path)
    df.to_sql(table_name, conn, if_exists="append", index=False)
    conn.close()
    print(f"Данные сохранены в SQLite таблицу '{table_name}' в {db_path}")


if __name__ == "__main__":
    ticker = "SBER"
    df = get_full_history(ticker)

    if not df.empty:
        save_to_csv(df, ticker)
        save_to_sqlite(df, table_name=ticker)
    else:
        print("Нет данных")


Данные сохранены в data/SBER.csv
Данные сохранены в SQLite таблицу 'SBER' в moex_data.db


In [61]:
import sqlite3
import pandas as pd

# Подключаемся к базе
conn = sqlite3.connect("moex_data.db")

# Смотрим первые строки таблицы
df = pd.read_sql("SELECT * FROM SBER LIMIT 10", conn)
print(df.head())

# Или посмотреть, какие вообще тикеры есть:
tickers = pd.read_sql("SELECT DISTINCT SECID FROM SBER", conn)
print(tickers)

  BOARDID   TRADEDATE SHORTNAME SECID  NUMTRADES         VALUE   OPEN    LOW  \
0    EQBR  2011-11-21  Сбербанк  SBER     128096  2.283014e+10  79.00  75.85   
1    SMAL  2011-11-21  Сбербанк  SBER         56  2.539949e+04  79.49  74.50   
2    EQBR  2011-11-22  Сбербанк  SBER     126144  2.091095e+10  76.40  75.10   
3    SMAL  2011-11-22  Сбербанк  SBER         50  8.413835e+04  73.20  73.20   
4    EQBR  2011-11-23  Сбербанк  SBER     140160  2.556061e+10  75.00  74.42   

    HIGH  LEGALCLOSEPRICE  ...  MARKETPRICE3  ADMITTEDQUOTE     MP2VALTRD  \
0  79.37            76.00  ...         77.71          77.71  2.283016e+10   
1  79.49            75.44  ...           NaN            NaN  0.000000e+00   
2  77.76            76.08  ...         76.71          76.71  2.091103e+10   
3  77.69            75.97  ...           NaN            NaN  0.000000e+00   
4  77.44            76.71  ...         76.21          76.21  2.556062e+10   

   MARKETPRICE3TRADESVALUE  ADMITTEDVALUE  WAVAL  TRADIN

In [2]:
import requests
import pandas as pd
from datetime import datetime
import time

def get_current_tickers():
    """Получить список текущих тикеров с Московской биржи"""
    url = "https://iss.moex.com/iss/engines/stock/markets/shares/securities.json"
    params = {
        'iss.meta': 'off',
        'iss.only': 'securities',
        'securities.columns': 'SECID,SHORTNAME,ISIN,LISTLEVEL,SECNAME,BOARDID'
    }
    
    response = requests.get(url, params=params)
    data = response.json()
    
    df = pd.DataFrame(data['securities']['data'], 
                     columns=data['securities']['columns'])
    return df

def get_historical_tickers():
    """Получить исторический список тикеров с Московской биржи"""
    # Получение исторических данных через API истории
    url = "https://iss.moex.com/iss/history/engines/stock/markets/shares/securities.json"
    
    all_data = []
    start = 0
    limit = 100
    
    while True:
        params = {
            'start': start,
            'limit': limit,
            'iss.meta': 'off',
            'history.columns': 'SECID,SHORTNAME,BOARDID,TRADEDATE'
        }
        
        print(f"Получение данных с позиции {start}...")
        response = requests.get(url, params=params)
        data = response.json()
        
        chunk = data['history']['data']
        all_data.extend(chunk)
        
        if len(chunk) < limit:
            break
            
        start += limit
        time.sleep(0.5)  # Пауза чтобы не перегружать API
    
    df = pd.DataFrame(all_data, columns=data['history']['columns'])
    return df

def scrape_from_listing_history():
    """Получить данные о листинге и делистинге с истории"""
    # Точных URL нужно уточнить, это примерный подход
    url = "https://www.moex.com/s19"
    # Здесь требуется web scraping
    # Код будет зависеть от структуры страницы
    
    # Пример логики:
    # response = requests.get(url)
    # soup = BeautifulSoup(response.text, 'html.parser')
    # data = extract_listing_history(soup)
    # return pd.DataFrame(data)
    pass

def export_merged_data():
    """Объединить данные из разных источников и выполнить экспорт"""
    # Получение текущих тикеров
    current_df = get_current_tickers()
    current_df['status'] = 'active'
    
    # Получение исторических тикеров
    historical_df = get_historical_tickers()
    
    # Объединение и удаление дубликатов
    historical_unique = historical_df.drop_duplicates(subset=['SECID'])
    
    # Находим тикеры, которых нет в текущем списке
    current_tickers = set(current_df['SECID'])
    delisted = historical_unique[~historical_unique['SECID'].isin(current_tickers)]
    delisted['status'] = 'delisted'
    
    # Объединяем активные и делистингованные акции
    all_tickers = pd.concat([current_df, delisted])
    
    # Запись в CSV
    timestamp = datetime.now().strftime('%Y%m%d')
    all_tickers.to_csv(f'moex_all_tickers_{timestamp}.csv', index=False)
    
    # Вернуть также DataFrame с результатами
    return all_tickers

if __name__ == "__main__":
    print("Начинаем сбор данных о тикерах Московской биржи...")
    tickers_df = export_merged_data()
    print(f"Готово! Собрано {len(tickers_df)} тикеров.")
    print(f"Активных: {len(tickers_df[tickers_df['status'] == 'active'])}")
    print(f"Делистингованных: {len(tickers_df[tickers_df['status'] == 'delisted'])}")

Начинаем сбор данных о тикерах Московской биржи...
Получение данных с позиции 0...
Получение данных с позиции 100...
Получение данных с позиции 200...
Получение данных с позиции 300...
Получение данных с позиции 400...
Готово! Собрано 684 тикеров.
Активных: 684
Делистингованных: 0


In [13]:
df = pd.read_csv(
    r"C:\Users\sofya\Desktop\Export_ru_securities-list_20250405.csv",
    encoding='utf-8',  # кодировка Windows, часто используется в России
    sep=';',            # разделитель — точка с запятой
    on_bad_lines='skip' # игнорируем строки с ошибками
)

df.head()


,NPP,EMITENT_FULL_NAME,TRADE_CODE,LIST_SECTION,SUPERTYPE,INSTRUMENT_TYPE,INSTRUMENT_CATEGORY,ISIN,REGISTRY_NUMBER,REGISTRY_DATE,INN,NOMINAL,CURRENCY,ISSUE_AMOUNT,DECISION_DATE,LISTING_LEVEL_HIST,COUPON_PERCENT,DISCLOSURE_PART_PAGE,DISCLOSURE_RF_INFO_PAGE,INCLUDE_DATE
0,1,"""Газпромбанк"" (Акционерное общество)",NaN,Первый уровень,Облигации,Облигация биржевая,биржевые облигации процентные неконвертируемые...,NaN,4B02-02-00354-B-003P,04.06.2021 0:00,7.744001e+09,1000,Рубль,10000000,04.06.2021 0:00,04.06.2021 Включение в Первый уровень,NaN,https://www.gazprombank.ru,https://www.e-disclosure.ru/portal/company.asp...,04.06.2021 0:00
1,2,"""Газпромбанк"" (Акционерное общество)",NaN,Первый уровень,Облигации,Облигация биржевая,биржевые облигации процентные неконвертируемые...,NaN,4B02-03-00354-B-003P,04.06.2021 0:00,7.744001e+09,1000,Рубль,10000000,04.06.2021 0:00,04.06.2021 Включение в Первый уровень,NaN,https://www.gazprombank.ru,https://www.e-disclosure.ru/portal/company.asp...,04.06.2021 0:00
2,3,"""Газпромбанк"" (Акционерное общество)",NaN,Первый уровень,Облигации,Облигация биржевая,биржевые облигации процентные неконвертируемые...,NaN,4B02-04-00354-B-003P,04.06.2021 0:00,7.744001e+09,1000,Рубль,10000000,04.06.2021 0:00,04.06.2021 Включение в Первый уровень,NaN,https://www.gazprombank.ru,https://www.e-disclosure.ru/portal/company.asp...,04.06.2021 0:00
3,4,"""Газпромбанк"" (Акционерное общество)",NaN,Первый уровень,Облигации,Облигация биржевая,биржевые облигации процентные неконвертируемые...,NaN,4B02-05-00354-B-003P,04.06.2021 0:00,7.744001e+09,1000,Рубль,10000000,04.06.2021 0:00,04.06.2021 Включение в Первый уровень,NaN,https://www.gazprombank.ru,https://www.e-disclosure.ru/portal/company.asp...,04.06.2021 0:00
4,5,"""Газпромбанк"" (Акционерное общество)",RU000A101Z74,Первый уровень,Облигации,Облигация биржевая,биржевые облигации процентные неконвертируемые...,RU000A101Z74,4B02-16-00354-B-001P,24.07.2020 0:00,7.744001e+09,1000,Рубль,10630210,24.07.2020 0:00,24.07.2020 Включение в Первый уровень,"12,75%",https://www.gazprombank.ru,https://www.e-disclosure.ru/portal/company.asp...,24.07.2020 0:00


In [15]:
df_stocks = df[df['SUPERTYPE'] == 'Акции']
df_stocks

,NPP,EMITENT_FULL_NAME,TRADE_CODE,LIST_SECTION,SUPERTYPE,INSTRUMENT_TYPE,INSTRUMENT_CATEGORY,ISIN,REGISTRY_NUMBER,REGISTRY_DATE,INN,NOMINAL,CURRENCY,ISSUE_AMOUNT,DECISION_DATE,LISTING_LEVEL_HIST,COUPON_PERCENT,DISCLOSURE_PART_PAGE,DISCLOSURE_RF_INFO_PAGE,INCLUDE_DATE
11,12,"""МОСКОВСКИЙ КРЕДИТНЫЙ БАНК"" (публичное акционе...",CBOM,Первый уровень,Акции,Акция обыкновенная,акции обыкновенные,RU000A0JUG31,10101978B,18.08.1999 0:00,7.734203e+09,1,Рубль,33429709866,11.03.2014 0:00,22.06.2015 Включение в Первый уровень 09.06.20...,NaN,http:// www.mkb.ru,http://www.e-disclosure.ru/portal/company.aspx...,22.06.2015 0:00
22,23,"Акционерная компания ""АЛРОСА"" (публичное акцио...",ALRS,Первый уровень,Акции,Акция обыкновенная,акции обыкновенные,RU0007252813,1-03-40046-N,25.08.2011 0:00,1.433000e+09,"0,5",Рубль,7364965630,23.11.2011 0:00,09.06.2014 Включение в Первый уровень 02.10.20...,NaN,http://www.alrosa.ru/инвесторам-и-акционерам/,https://e-disclosure.ru/portal/company.aspx?id...,09.06.2014 0:00
104,105,Банк ВТБ (публичное акционерное общество),VTBR,Первый уровень,Акции,Акция обыкновенная,акции обыкновенные,RU000A0JP5V6,10401000B,29.09.2006 0:00,7.702070e+09,50,Рубль,5369933893,28.04.2007 0:00,09.06.2014 Включение в Первый уровень 28.02.20...,NaN,http://www.vtb.ru,http://www.e-disclosure.ru/portal/company.aspx...,09.06.2014 0:00
134,135,Международная Компания Публичное Акционерное О...,MDMG,Первый уровень,Акции,Акция обыкновенная,акции обыкновенные,RU000A108KL3,1-01-16802-A,13.05.2024 0:00,3.900026e+09,"7,16872",Рубль,75125010,05.11.2020 0:00,05.11.2020 Включение в Первый уровень,NaN,https://www.mcclinics.ru/,https://e-disclosure.ru/portal/company.aspx?id...,05.11.2020 0:00
135,136,Международная Компания Публичное Акционерное О...,GEMC,Первый уровень,Акции,Акция обыкновенная,акции обыкновенные,RU000A107JE2,1-01-16774-A,18.12.2023 0:00,3.900020e+09,"0,03721404",Рубль,90000000,06.07.2021 0:00,15.07.2021 Включение в Первый уровень,NaN,https://www.emcinvestors.ru/,https://www.e-disclosure.ru/portal/company.asp...,15.07.2021 0:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3422,3423,Публичное акционерное общество энергетики и эл...,SAGOP,Третий уровень,Акции,Акция привилегированная,акции привилегированные,RU0009084495,2-02-00127-A,02.11.2006 0:00,6.315223e+09,"0,01",Рубль,521993080,14.10.2011 0:00,09.06.2014 Включение в Третий уровень 14.10.20...,NaN,NaN,http://disclosure.1prime.ru/portal/default.asp...,09.06.2014 0:00
3423,3424,Публичное акционерное общество энергетики и эл...,SLEN,Третий уровень,Акции,Акция обыкновенная,акции обыкновенные,RU0009280465,1-03-00272-A,08.04.2008 0:00,6.500000e+09,10,Рубль,1237175444,31.01.2017 0:00,31.01.2017 Включение в Третий уровень,NaN,http://sakhalinenergo.ru/aktsioneram-i-investo...,http://www.e-disclosure.ru/portal/company.aspx...,31.01.2017 0:00
3466,3467,"акционерный коммерческий банк ""Приморье"" (пуб...",PRMB,Третий уровень,Акции,Акция обыкновенная,акции обыкновенные,RU000A0DPNQ5,10103001B,09.08.1994 0:00,2.536021e+09,1000,Рубль,250000,13.01.2005 0:00,09.06.2014 Включение в Третий уровень 13.01.20...,NaN,NaN,https://www.e-disclosure.ru/portal/company.asp...,09.06.2014 0:00
3567,3568,"публичное акционерное общество ""Ковровский мех...",KMEZ,Третий уровень,Акции,Акция обыкновенная,акции обыкновенные,RU0006753613,1-02-05376-A,30.06.2009 0:00,3.305004e+09,250,Рубль,3764600,27.12.2012 0:00,09.06.2014 Включение в Третий уровень 28.12.20...,NaN,http://www.kvmz.ru/raskrytie-informatsii/,http://www.e-disclosure.ru/portal/company.aspx...,09.06.2014 0:00
